### Trying to run the dense indexing for the MS MARCO dataset

The sparse index that we will use for our pipeline:

In [1]:
import pyterrier as pt
from experiments_helper import time_fct

if not pt.started():
    pt.init()

PyTerrier 0.10.0 has loaded Terrier 5.9 (built by craigm on 2024-05-02 17:40) and terrier-helper 0.0.8

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


In [2]:
from pyterrier.measures import RR, nDCG, MAP

eval_metrics = [RR @ 10, nDCG @ 10, MAP @ 100]

In [3]:
from experiments_helper import run_single_experiment_name

index_absolute_path = "/home/anistor/.pyterrier/corpora/msmarco_passage/index/terrier_stemmed"


index = pt.IndexFactory.of(index_absolute_path)

bm25_stored = pt.BatchRetrieve(index, wmodel="BM25")

dataset_name = "msmarco_passage"

dataset_test_name = "irds:msmarco-passage/trec-dl-2019"
time_fct(run_single_experiment_name, bm25_stored, dataset_test_name, eval_metrics, dataset_name + ": BM25")


Experiment took 12.220 seconds to execute.


,name,RR@10,nDCG@10,AP@100
0,msmarco_passage: BM25,0.79438,0.47954,0.290692


Initialise the Query and Document encoders for GTE base

In [6]:
from pathlib import Path
from gte_base_en_encoder import GTEBaseDocumentEncoder
import torch
from fast_forward import OnDiskIndex, Mode, Indexer

q_encoder = GTEBaseDocumentEncoder("Alibaba-NLP/gte-base-en-v1.5")
d_encoder = GTEBaseDocumentEncoder(
    "Alibaba-NLP/gte-base-en-v1.5",
    device="cuda:0" if torch.cuda.is_available() else "cpu",
)

Define a function that will iterate only the first 1000 docs in order to get an understanding of how much time it would take

In [7]:
def docs_iter(dataset, limit=1000):
    # Iterate over the documents and yield up to 'limit' documents
    for count, d in enumerate(dataset.get_corpus_iter()):
        if count >= limit:
            break
        yield {"doc_id": d["docno"], "text": d["text"]}

Function that formats the dataset name to avoid ":" in the index filename

In [8]:
def format_dataset_name(text):
    if ':' in text:
        return text.split(':')[0] + "_" + text.split(':')[1]
    else:
        return text

The passage that I am willing to index. I assumed that the sparse and dense indexes must embed the exact same dataset, "msmarco_passage"

In [9]:
dataset_name = "msmarco_passage"
dataset = pt.get_dataset(dataset_name)

Create indexer and start indexing. I get an error and my understanding is that the files is not available. However, i don't understand the reason for this error as the dataset appears as available on the Pyterrier website(https://pyterrier.readthedocs.io/en/latest/datasets.html#:~:text=Deep%2DLearning.html-,msmarco_passage,-True).

In [10]:
index_path = "../dense_indexes/ffindex_" + dataset_name + "_gte-base-en-v1.5.h5"
ff_index = OnDiskIndex(
    Path(index_path), dim=768, query_encoder=q_encoder, mode=Mode.MAXP, max_id_length=47
)

In [11]:
ff_indexer = Indexer(ff_index, d_encoder, batch_size=8)
ff_indexer.index_dicts(docs_iter(dataset))

0it [00:00, ?it/s]

0it [00:00, ?it/s]


HTTPError: 404 Client Error: The specified resource does not exist. for url: https://msmarco.blob.core.windows.net/msmarcoranking/collection.tar.gz

I chose the "irds" version of the MC MARCO passage as I have only used irds previously for the BEIR set and I wanted to see if it works. I started indexing and it seemed that it would work. However, I did not know is it is ok to run BM25 on msmarco-passage and the dense index and the test on "irds:msmarco-passage". I would like to use the msmarco-passage pre-built index to avoid creating a sparse index for this dataset. Is the previous approach(runnning BM25 on msmarco-passage and the dense index and the testing on "irds:msmarco-passage") a good approach or I should avoid it?

In [12]:
dataset_name = "irds:msmarco-passage"
dataset = pt.get_dataset(dataset_name)

In [13]:

index_path = "../dense_indexes/ffindex_" + format_dataset_name(dataset_name) + "_gte-base-en-v1.5.h5"
ff_index = OnDiskIndex(
    Path(index_path), dim=768, query_encoder=q_encoder, mode=Mode.MAXP, max_id_length=47
)

In [14]:
ff_indexer = Indexer(ff_index, d_encoder, batch_size=8)
ff_indexer.index_dicts(docs_iter(dataset))

0it [00:00, ?it/s]
                  cuments:   0%|          | 0/8841823 [00:00<?, ?it/s]
                                                                   [INFO] Please confirm you agree to the MSMARCO data usage agreement found at <http://www.msmarco.org/dataset.aspx>
0it [00:00, ?it/s]
                  cuments:   0%|          | 0/8841823 [00:00<?, ?it/s]
                                                                   [INFO] If you have a local copy of https://msmarco.z22.web.core.windows.net/msmarcoranking/collectionandqueries.tar.gz, you can symlink it here to avoid downloading it again: /home/anistor/.ir_datasets/downloads/31644046b18952c1386cd4564ba2ae69
0it [00:00, ?it/s]
                  cuments:   0%|          | 0/8841823 [00:00<?, ?it/s]
                                                                   [INFO] [starting] https://msmarco.z22.web.core.windows.net/msmarcoranking/collectionandqueries.tar.gz
0it [00:00, ?it/s]
msmarco-passage documents:   0%|          | 0/88

KeyboardInterrupt: 